In [4]:
import numpy as np

class CNN:
    def conv_forward_2d(self, input, kernel, stride=1, padding=0):
        H, W = input.shape
        F, x = kernel.shape
        # Apply padding
        padded_input = np.pad(input, ((padding, padding), (padding, padding)), mode='constant')
        # print(padded_input)
        # Calculate output dimensions
        out_height = ((H - F + 2 * padding) // stride) + 1
        out_width = ((W - F + 2 * padding) // stride) + 1

        # Perform convolution
        out = np.zeros((out_height, out_width)) #declaring the 2d numpy array for output.
        for h in range(out_height):
            for w in range(out_width):
                h_start, w_start = h * stride, w * stride
                h_end, w_end = h_start + F, w_start + F
                out[h, w] = np.sum(padded_input[h_start:h_end, w_start:w_end] * kernel) #Calculating the cell in feature map

        return out

    def pool_forward_2d(self, input, pool_size, stride):
        H, W = input.shape
        pool_height, pool_width = pool_size

        # Calculate output dimensions using below formulaes
        out_height = (H - pool_height) // stride + 1
        out_width = (W - pool_width) // stride + 1

        # Perform pooling
        out = np.zeros((out_height, out_width)) #declaring the 2d numpy array for output.
        for h in range(out_height):
            for w in range(out_width):
                h_start, w_start = h * stride, w * stride
                h_end, w_end = h_start + pool_height, w_start + pool_width
                out[h, w] = np.max(input[h_start:h_end, w_start:w_end]) #using max pooling.

        return out
    def conv_backward_2d(self, d_out, input, kernel, stride=1, padding=1):
        H, W = input.shape
        F, x = kernel.shape

        padded_input = np.pad(input, ((padding, padding), (padding, padding)), mode='constant')
        d_input = np.zeros_like(input) #gradient w.r.t original input
        d_kernel = np.zeros_like(kernel) #gradient w.r.t to kernel

        # Add padding 
        d_padded_input = np.pad(d_input, ((padding, padding), (padding, padding)), mode='constant')
        #updating the above defined gradients
        for h in range(d_out.shape[0]):
            for w in range(d_out.shape[1]):
                h_start, w_start = h * stride, w * stride
                h_end, w_end = h_start + F, w_start + F
                # Gradient with respect to input
                d_padded_input[h_start:h_end, w_start:w_end] += kernel * d_out[h, w]
                # Gradient with respect to kernel
                d_kernel += padded_input[h_start:h_end, w_start:w_end] * d_out[h, w]

        # Remove padding from the gradient with respect to input
        if padding != 0:
            d_input = d_padded_input[padding:-padding, padding:-padding]
        else:
            d_input = d_padded_input

        return d_input, d_kernel

    def pool_backward_2d(self, d_out, input, pool_size, stride):
        H, W = input.shape
        pool_height, pool_width = pool_size

        d_input = np.zeros_like(input)# gradient w.r.t original inputs

        for h in range(d_out.shape[0]):
            for w in range(d_out.shape[1]):
                h_start, w_start = h * stride, w * stride
                h_end, w_end = h_start + pool_height, w_start + pool_width
                patch = input[h_start:h_end, w_start:w_end]
                max_value = np.max(patch) #extract max value

                # Place the gradient in the correct location
                for i in range(pool_height):
                    for j in range(pool_width):
                        if patch[i, j] == max_value:
                            d_input[h_start + i, w_start + j] = d_out[h, w]

        return d_input


cnn = CNN()
input_data = np.random.rand(6,6)
kernel_data = np.random.randn(5,5)
# Sample input data and kernel
# input_data = np.array([[1, 6, 2], [5, 3, 1], [7, 0, 4]], dtype=float)
# kernel_data = np.array([[1, 2], [-1, 0]], dtype=float)

# Forward pass
convoluted_output = cnn.conv_forward_2d(input_data, kernel_data)
pool_size = (1, 1)
stride = 1
pooled_output = cnn.pool_forward_2d(convoluted_output, pool_size, stride)

d_out_conv = np.random.rand(*convoluted_output.shape) #The gradient of the loss with respect to the output of the convolution layer.
d_out_pool = np.random.rand(*pooled_output.shape)

# Backward pass
d_input_conv, d_kernel_conv = cnn.conv_backward_2d(d_out_conv, input_data, kernel_data)
d_input_pool = cnn.pool_backward_2d(d_out_pool, convoluted_output, pool_size, stride)
print("Input:\n",input_data)
print("\nKernel:\n", kernel_data)
print("\nConvoluted Output:\n", convoluted_output)
print("\nPooled Output:\n", pooled_output)
print("\nGradient wrt Input of Convolution Layer:\n", d_input_conv)
print("\nGradient wrt Kernel of Convolution Layer:\n", d_kernel_conv)
print("\nGradient wrt Input of Pooling Layer:\n", d_input_pool)


Input:
 [[0.0794059  0.92367493 0.43787835 0.81333271 0.51651823 0.09850467]
 [0.96708127 0.87338758 0.27649303 0.21545931 0.06085395 0.95977758]
 [0.59290121 0.18855192 0.49817963 0.4980895  0.04520311 0.46983905]
 [0.60719807 0.45602215 0.85223572 0.14063419 0.60431076 0.31593875]
 [0.77940636 0.39593495 0.00462042 0.44176037 0.42108853 0.53754588]
 [0.9918958  0.34237261 0.92831006 0.41349058 0.6670381  0.76123018]]

Kernel:
 [[-8.69058504e-01 -5.96897237e-02 -8.73251311e-01 -1.38554435e-01
  -1.87291373e-04]
 [-4.79196670e-01 -1.09387936e+00 -5.43690442e-01 -2.34982687e-02
   1.02987232e+00]
 [ 5.36859442e-02  9.24942414e-01 -2.20802921e-01 -4.23817445e-01
   9.86260577e-01]
 [ 5.74697879e-01 -3.11859529e-01  1.29751232e+00  5.17936243e-01
   1.20349945e+00]
 [ 6.80090038e-01 -3.11211484e-01  4.77241430e-01  1.51500034e-01
   2.14290344e-01]]

Convoluted Output:
 [[0.47775353 0.87300845]
 [1.195016   1.5113761 ]]

Pooled Output:
 [[0.47775353 0.87300845]
 [1.195016   1.5113761 ]]



In [ ]:
import numpy as np
#In three dimension
def conv_forward(input, kernel, stride=1, padding=1):
    H, W, C = input.shape
    F, x, y = kernel.shape

    # Apply padding
    padded_input = np.pad(input, ((padding, padding), (padding, padding), (0, 0)), mode='constant')
    print(padded_input)
    # Calculate output dimensions
    out_height = (H - F + 2 * padding) // stride + 1
    out_width = (W - F + 2 * padding) // stride + 1

    # Perform convolution
    out = np.zeros((out_height, out_width, C))
    for h in range(out_height):
        for w in range(out_width):
            h_start, w_start = h * stride, w * stride
            h_end, w_end = h_start + F, w_start + F
            out[h, w, :] = np.sum(padded_input[h_start:h_end, w_start:w_end, :] * kernel, axis=(0, 1))
    
    return out

input_example = np.random.rand(5, 5, 3)
print(input_example)
kernel_example = np.random.rand(3, 3, 3)
conv_out = conv_forward(input_example, kernel_example)
print("Convolution Output:\n", conv_out)



In [ ]:
import numpy as np


def conv_backward(dout, input, kernel, stride=1, padding=1):
    H, W, C = input.shape
    F, _, _ = kernel.shape
    out_height, out_width, _ = dout.shape

    # Apply padding
    padded_input = np.pad(input, ((padding, padding), (padding, padding), (0, 0)), mode='constant')
    dpadded_input = np.zeros_like(padded_input)
    
    # Perform convolution backward pass
    for h in range(out_height):
        for w in range(out_width):
            h_start, w_start = h * stride, w * stride
            h_end, w_end = h_start + F, w_start + F
            dpadded_input[h_start:h_end, w_start:w_end, :] += kernel * dout[h, w, np.newaxis, np.newaxis, :]

    # Remove padding
    # print(dpadded_input)
    dinput = dpadded_input[padding:-padding, padding:-padding, :]
    return dinput

# Example usage
input_example = np.random.rand(5, 5, 3)
# print(input_example)
kernel_example = np.random.rand(3, 3, 3)
dout_example = np.random.rand(3, 3, 3)
conv_backward_out = conv_backward(dout_example, input_example, kernel_example)
print("Convolution Backward Output:\n", conv_backward_out)


In [ ]:
import numpy as np


def pool_forward(input, pool_size, stride):
    H, W, C = input.shape
    pool_height, pool_width = pool_size

    # Calculate output dimensions
    out_height = (H - pool_height) // stride + 1
    out_width = (W - pool_width) // stride + 1

    # Perform pooling
    out = np.zeros((out_height, out_width, C))
    for h in range(out_height):
        for w in range(out_width):
            h_start, w_start = h * stride, w * stride
            h_end, w_end = h_start + pool_height, w_start + pool_width
            out[h, w, :] = np.max(input[h_start:h_end, w_start:w_end, :], axis=(0, 1))

    return out
input_example = np.random.rand(5, 5, 3)
pool_out = pool_forward(input_example, pool_size=(2, 2), stride=2)
print("Pooling Output:\n", pool_out)


In [ ]:
import numpy as np

def pool_backward(dout, input, pool_size, stride):
    H, W, C = input.shape
    pool_height, pool_width = pool_size
    out_height, out_width, _ = dout.shape
    dinput = np.zeros_like(input)

    for h in range(out_height):
        for w in range(out_width):
            h_start, w_start = h * stride, w * stride
            h_end, w_end = h_start + pool_height, w_start + pool_width
            pool_region = input[h_start:h_end, w_start:w_end, :]
            max_pool_indices = pool_region == np.max(pool_region, axis=(0, 1), keepdims=True)
            dinput[h_start:h_end, w_start:w_end, :] += max_pool_indices * dout[h, w, np.newaxis, np.newaxis, :]

    return dinput

dout_example = np.random.rand(3, 3, 3)
input_example = np.random.rand(5, 5, 3)
pool_backward_out = pool_backward(dout_example, input_example, pool_size=(2, 2), stride=2)
print("Pooling Backward Output:\n", pool_backward_out)
